In [ ]:
import pandas as pd
from scipy.stats import zscore
from datetime import datetime

file_ahl = pd.read_csv('2021-2022_AHL_all_players.csv')
file_ahl_ep = pd.read_csv('2021-2022_AHL_players_EP.csv', encoding= 'unicode_escape')
file_nhl = pd.read_csv('2021-2022_player_stats_naturalstattrick.csv')
df_name_update = pd.read_csv('2021-2022_name_updates.csv')

min_rate = 54
max_rate = 68
rate_range = max_rate - min_rate

#### Merge Stats and EP data

In [ ]:
file_ahl.sort_values(by=['Player','GP'],ascending=False,inplace=True)
file_ahl.drop_duplicates(subset='Player',keep='first',inplace=True)
file_ahl_ep['Player'] = file_ahl_ep['Player'].str.strip(' ')
file_ahl_merge = file_ahl.merge(file_ahl_ep,on='Player',how='left')

# Players with more AHL games than NHL games

In [ ]:
nhl = df_name_update['NHL Name'].to_list()
ahl = df_name_update['AHL Name'].to_list()

file_ahl_merge['Player'] = file_ahl_merge['Player'].replace(ahl,nhl)

In [ ]:
df_ahl_init_cl = file_ahl_merge.loc[:,['Player','GP']]
df_nhl_init_cl = file_nhl.loc[:,['Player','GP']]

df_ahl_init_cl['League'] = 'AHL'
df_nhl_init_cl['League'] = 'NHL'

df_names = pd.concat([df_nhl_init_cl,df_ahl_init_cl]).reset_index(drop=True)
df_names['Player'] = df_names['Player'].str.lower()

df_names = df_names.iloc[df_names.groupby('Player')['GP'].idxmax()]

df_ahl_names = df_names[df_names['League'] == 'AHL'].values.tolist()
names_list = [name[0] for name in df_ahl_names]

file_ahl_merge['Player'] = file_ahl_merge['Player'].str.lower()
file_clean = file_ahl_merge[file_ahl_merge['Player'].isin(names_list)]

file_clean

In [ ]:
def get_rating(rating,stat1,stat2,stat1_wgt,stat2_wgt,inv):
    df = file_clean.loc[:,['Player',stat1,stat2]]
    
    df[f'{stat1}_zscore'] = zscore(df[stat1])
    df[f'{stat1}_zscore'] = df[f'{stat1}_zscore'] * inv
    z_max = df[f'{stat1}_zscore'].max()
    z_min = df[f'{stat1}_zscore'].min()
    z_range = z_max-z_min
    z_incr = z_range/rate_range

    df[f'{stat1}_Rating'] = max_rate-((z_max-df[f'{stat1}_zscore'])/z_incr)
    
    df[f'{stat2}_zscore'] = zscore(df[stat2])
    df[f'{stat2}_zscore'] = df[f'{stat2}_zscore'] * inv
    z_max2 = df[f'{stat2}_zscore'].max()
    z_min2 = df[f'{stat2}_zscore'].min()
    z_range2 = z_max2-z_min2
    z_incr2 = z_range2/rate_range

    df[f'{stat2}_Rating'] = max_rate-((z_max2-df[f'{stat2}_zscore'])/z_incr2)
    df[rating] = (df[f'{stat1}_Rating'] * stat1_wgt) + (df[f'{stat2}_Rating'] * stat2_wgt)
    df_rate = df.loc[:,['Player',rating]]
    df_rate.sort_values(by=rating,ascending=False,inplace=True)
    
    return df_rate

# CK

In [ ]:
ck = get_rating('CK','PIM/G','GP',1,0,1)

# FG

In [ ]:
fg = get_rating('FG','FG','GP',1,0,1)

# DI

In [ ]:
di = get_rating('DI','PIM','GP',1,0,-1)

# SK

In [ ]:
sk = get_rating('SK','Shots','GP',1,0,1)

# ST

In [ ]:
st_max = 80
st_min = 50
st_range = st_max - st_min

st_calc = file_clean.loc[:,['Player','Pos','height','weight']]
st_calc['height'].fillna(180,inplace=True)
st_calc['weight'].fillna(190,inplace=True)
st_calc['height'] = round(st_calc['height'] / 2.54).astype(int)
st_calc['ratio'] = (st_calc['height'] * 0.75) + (st_calc['weight'] * 0.25)


st_calc['ratio_zscore'] = zscore(st_calc['ratio'])
z_max = st_calc['ratio_zscore'].max()
z_min = st_calc['ratio_zscore'].min()
z_range = z_max-z_min
z_incr = z_range/st_range

st_calc['ST'] = st_max-((z_max-st_calc['ratio_zscore'])/z_incr)
st = st_calc.loc[:,['Player','Pos','height','weight','ST']]

st.sort_values(by='ST',ascending=False)

# EN

In [ ]:
en = get_rating('EN','Shots','GP',0,1,1)

# DU

In [ ]:
du = get_rating('DU','PIM','GP',0.2,0.8,1)

# PH

In [ ]:
ph = get_rating('PH','PTS','GP',1,0,1)

# FO

In [ ]:
fo = file_clean.loc[:,['Player','Pos']]

fo['Pos'].replace({'C' : 70,'LW': 55, 'RW' : 55, 'F' : 55,'D' : 40}, inplace=True)

fo.rename(columns={'Pos': 'FO'}, inplace=True)

# PA

In [ ]:
pa = get_rating('PA','A','GP',1,0,1)

# SC

In [ ]:
sc = get_rating('SC','G','GP',1,0,1)

# DF

In [ ]:
df = get_rating('DF',' +/-','GP',1,0,1)

# PS

In [ ]:
ps = get_rating('PS','SOG','GP',1,0,1)

# EX

In [ ]:
ex_max = 85
ex_min = 50
ex_range = ex_max - ex_min

ex_calc = file_clean.loc[:,['Player','nation','dob','link']]
ex_calc['Year'] = ex_calc['dob'].str[:4]
ex_calc['Year'].fillna(2000,inplace=True)
ex_calc['Month'] = ex_calc['dob'].str[5:7]
ex_calc['Day'] = ex_calc['dob'].str[8:]
ex_calc['Age'] = 2021 - ex_calc['Year'].astype(int)

ex_calc['ex_zscore'] = zscore(ex_calc['Age'])
z_ex_max = ex_calc['ex_zscore'].max()
z_ex_min = ex_calc['ex_zscore'].min()
z_ex_range = z_ex_max-z_ex_min
z_ex_incr = z_ex_range/ex_range

ex_calc['EX'] = ex_max-((z_ex_max-ex_calc['ex_zscore'])/z_ex_incr)
ex = ex_calc.loc[:,['Player','nation','Year','Month','Day','link','EX']]

ex.sort_values(by='EX',ascending=False)

# LD

In [ ]:
ld_max = 70
ld_min = 50
ld_range = ld_max - ld_min

ld_calc = file_clean.loc[:,['Player','dob']]
ld_calc['Year'] = ld_calc['dob'].str[:4]
ld_calc['Year'].fillna(2000,inplace=True)
ld_calc['Age'] = 2021 - ld_calc['Year'].astype(int)

ld_calc['ld_zscore'] = zscore(ld_calc['Age'])
z_ld_max = ld_calc['ld_zscore'].max()
z_ld_min = ld_calc['ld_zscore'].min()
z_ld_range = z_ld_max-z_ld_min
z_ld_incr = z_ld_range/ld_range

ld_calc['LD'] = ld_max-((z_ld_max-ld_calc['ld_zscore'])/z_ld_incr)
ld = ld_calc.loc[:,['Player','LD']]

ld.sort_values(by='LD',ascending=False)

# OV

In [ ]:
df_ov = ck.merge(fg, how='left',on=['Player']).merge(di, how='left',on=['Player']).merge(sk, how='left',on=['Player']).merge(st, how='left',on=['Player']).merge(en, how='left',on=['Player']).merge(du, how='left',on=['Player']).merge(ph, how='left',on=['Player']).merge(fo, how='left',on=['Player']).merge(pa, how='left',on=['Player']).merge(sc, how='left',on=['Player']).merge(df, how='left',on=['Player']).merge(ps, how='left',on=['Player']).merge(ex, how='left',on=['Player']).merge(ld, how='left',on=['Player'])
df_ov['Rookie'] = 'N'
df_ov['PO'] = 1
df_ov['Contract'] = 1
df_ov['Salary'] = 1
df_ov['League'] = 'AHL'

df_ov.drop_duplicates(subset='Player',inplace=True)
df_ov_f = df_ov[df_ov['Pos'] != 'D']
df_ov_d = df_ov[df_ov['Pos'] == 'D']

df_ov_d['OV'] = (df_ov_d['CK'] *0.08) + (df_ov_d['DI'] *0.02) + (df_ov_d['SK'] *0.10) + (df_ov_d['ST'] *0.05) + (df_ov_d['EN'] *0.09) + (df_ov_d['DU'] *0.02) + (df_ov_d['PH'] *0.14) + (df_ov_d['PA'] *0.12) + (df_ov_d['SC'] *0.16) + (df_ov_d['DF'] *0.15) + (df_ov_d['EX'] *0.03) + (df_ov_d['LD'] *0.04)
df_ov_f['OV'] = (df_ov_f['CK'] *0.07) + (df_ov_f['DI'] *0.02) + (df_ov_f['SK'] *0.10) + (df_ov_f['ST'] *0.05) + (df_ov_f['EN'] *0.09) + (df_ov_f['DU'] *0.02) + (df_ov_f['PH'] *0.14) + (df_ov_f['PA'] *0.14) + (df_ov_f['SC'] *0.19) + (df_ov_f['DF'] *0.11) + (df_ov_f['EX'] *0.03) + (df_ov_f['LD'] *0.03) + (df_ov_f['FO'] *0.01)

df_ov_rate = pd.concat([df_ov_d,df_ov_f])
df_ov_rate = df_ov_rate[['Player','Pos','nation','Rookie','Year','Month','Day','weight','height','Contract','Salary','link','CK','FG','DI','SK','ST','EN','DU','PH','FO','PA','SC','DF','PS','EX','LD','PO','OV','League']]
df_ov_rate['Player'] = df_ov_rate['Player'].str.title()

df_to_print = df_ov_rate.sort_values(by=['OV'],ascending=False)
df_to_print.to_csv('AHL_Skater_Ratings_2021-2022.csv',index=False)

df_to_print